<a href="https://colab.research.google.com/github/sirpps/youtube-splitter/blob/main/split_audio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Youtube audio Splitter
`En:` An code to extract audio from long youtube video or livestream easly, requeriment Pixeldrain api key, tap  [here](https://pixeldrain.com/user/api_keys)<br>`Id:` Sebuah kode untuk mengekstrak audio dari video panjang youtube secara mudah, bahan yang diperlukan adalah Pixeldrain api key, klik [disini](https://pixeldrain.com/user/api_keys)

In [ ]:
#@title Install requirements
from IPython.display import clear_output as cls
import os
#============Colour=============
cyellow = "\033[33m"
cred = "\033[31m"
creset = "\033[0m"
cbold = "\033[1m"
citalic = "\033[3m"
!rm -rf *
!pip install yt-dlp
!pip install requests
cls()
print(cyellow+"[installer] "+creset+"yt-dlp version:", os.popen("yt-dlp --version").read().strip())
print(cyellow+"[installer] "+creset+"yt-dlp has been installed")

In [2]:
#@title Setup
#===========Main code===========
youtube_url = "" #@param{type:"string"}
audio_format = "wav" #@param["mp3","wav","opus","aac"]
#@markdown ---
#@markdown `En:` Input with format `hh:mm:ss` if the video under 1 hour just put 00 in the hour selection<br>Note: h is hour, m is minute, s is second<br>
#@markdown `Id:` Masukan dengan format `jj:mm:dd` jika video kurang dari 1 jam isi pada bagian jj 00<br>Catatan: j itu jam, m itu menit, d itu detik
#==========Making Path==========
start_time_h = "00" #@param ["00", "01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12", "13", "14", "15", "16", "17", "18", "19", "20", "21", "22", "23", "24"]
start_time_m = "00" #@param ["00", "01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12", "13", "14", "15", "16", "17", "18", "19", "20", "21", "22", "23", "24", "25", "26", "27", "28", "29", "30", "31", "32", "33", "34", "35", "36", "37", "38", "39", "40", "41", "42", "43", "44", "45", "46", "47", "48", "49", "50", "51", "52", "53", "54", "55", "56", "57", "58", "59"]
start_time_s = "00" #@param ["00", "01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12", "13", "14", "15", "16", "17", "18", "19", "20", "21", "22", "23", "24", "25", "26", "27", "28", "29", "30", "31", "32", "33", "34", "35", "36", "37", "38", "39", "40", "41", "42", "43", "44", "45", "46", "47", "48", "49", "50", "51", "52", "53", "54", "55", "56", "57", "58", "59"]
#@markdown ---
end_time_h = "00" #@param ["00", "01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12", "13", "14", "15", "16", "17", "18", "19", "20", "21", "22", "23", "24"]
end_time_m = "00" #@param ["00", "01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12", "13", "14", "15", "16", "17", "18", "19", "20", "21", "22", "23", "24", "25", "26", "27", "28", "29", "30", "31", "32", "33", "34", "35", "36", "37", "38", "39", "40", "41", "42", "43", "44", "45", "46", "47", "48", "49", "50", "51", "52", "53", "54", "55", "56", "57", "58", "59"]
end_time_s = "00" #@param ["00", "01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12", "13", "14", "15", "16", "17", "18", "19", "20", "21", "22", "23", "24", "25", "26", "27", "28", "29", "30", "31", "32", "33", "34", "35", "36", "37", "38", "39", "40", "41", "42", "43", "44", "45", "46", "47", "48", "49", "50", "51", "52", "53", "54", "55", "56", "57", "58", "59"]
#@markdown ---
#@markdown Enter [pixeldrain](https://pixeldrain.com/user/api_keys) api key
pixeldrain_apikey = "" #@param{type:"string"}
start_time = f"{start_time_h}:{start_time_m}:{start_time_s}"
end_time = f"{end_time_h}:{end_time_m}:{end_time_s}"

In [ ]:
#@title Downloading and extract audio
print("[youtube] Downloading yt audio")
!yt-dlp -q --progress -f bestaudio --prefer-ffmpeg -x --audio-format {audio_format} {youtube_url}
print("[youtube] Downloading done")
print("[youtube] Renaming file")
!mv *.{audio_format} aud_raw.{audio_format}
print("[youtube] Done All")

In [ ]:
#@title Triming the audio
if start_time != "00:00:00" and end_time == "00:00:00":
    print(cyellow + "[log] " + creset + "en: end time is zero, please fill the form")
    print(cyellow + "[log] " + creset + "id:" + citalic + cbold + " end timenya di isi goblok, lu mau motong apa endnya nol" + creset)
elif start_time == "00:00:00" and end_time == "00:00:00":
    print(cyellow + "[log] " + creset + "en: start and end time is zero, downloading full vid")
    print(cyellow + "[log] " + creset + "id:" + citalic + cbold + " waktu start dan end adalah 0, mengunduh semua")
    !mv *.{audio_format} aud_out.{audio_format}
else:
    print(cyellow + "[log] " + creset + "Starting")
    !ffmpeg -i *.{audio_format} -ss {start_time} -to {end_time} -c copy aud_out.{audio_format}
    cls()
    print(cyellow + "[trimer] " + creset + "Done")

In [ ]:
#@title Renaming audio file
import subprocess
import json

vidtitel = subprocess.run(
    ['yt-dlp', '--dump-json', youtube_url],
    capture_output=True,
    text=True
)
try:
    titellama = json.loads(vidtitel.stdout)['title']
    titel = titellama.lower().replace("-","").replace("  "," ").replace(" ","-").replace("(","").replace(")","")
    titelbaru = f"{titel}.{audio_format}"
    !mv aud_out.{audio_format} {titelbaru}
    print(cyellow+"[renamer] "+creset+"done")
except json.JSONDecodeError:
        print(cred+"An Error Occurred"+creset)

In [ ]:
#@title Uploading to [Pixeldrain](https://pixeldrain.com/)

upload = subprocess.run(
    ['curl', '-T', titelbaru, '-u', ':'+pixeldrain_apikey, 'https://pixeldrain.com/api/file/'],
    capture_output=True,
    text=True
)
#====================Debug============
#print("Output from curl:", upload.stdout)
#print("Error from curl:", upload.stderr)
#====================Making Path============
try:
    file_result_id = json.loads(upload.stdout)['id']
    print(cyellow+"[uploader] "+creset+"Upload done")
    print("file link: https://pixeldrain.com/u/" + file_result_id)
#    print("ID file:", file_result_id)
except json.JSONDecodeError:
        print(cred+"An Error Occurred"+creset)